# 🌊 01. 시퀀셜 데이터 & RNN (Recurrent Neural Network)

## 1. 시퀀셜 데이터(Sequential Data)란?
**"순서(Sequence)"가 중요한 데이터**를 말합니다.
순서를 바꾸면 의미가 완전히 달라지거나 망가지는 데이터들이죠.

> **쉬운 비유 (도미노)**:  
> 도미노 하나가 쓰러지면, 그 다음 도미노도 쓰러집니다. **앞의 사건이 뒤에 영향을 미치는 것**, 이것이 시퀀셜 데이터의 핵심입니다.

### 예시
- **텍스트**: "너를 사랑해" vs "사랑해 너를" (순서가 바뀌면 어색함)
- **시계열(Time Series)**: 오늘의 주가는 어제 주가의 영향을 받음.
- **음성**: "아" 소리 다음에 "빠"가 와야 "아빠"가 됨.

---

## 2. RNN: 기억을 가진 신경망

기존의 신경망(Feed Forward)은 데이터를 하나 보고 잊어버렸습니다. (금붕어 기억력 🐟)
하지만 문장을 이해하려면 **앞 내용을 기억**하고 있어야 뒷 내용을 이해할 수 있죠.

> **쉬운 비유 (이어달리기/Relay Race)**: 🏃
> - 첫 번째 선수가 뛰고 나서, **바통(기억, Hidden State)**을 두 번째 선수에게 넘겨줍니다.
> - 두 번째 선수는 바통을 받아 계속 뜁니다.
> - 이렇게 **이전 단계의 정보(바통)를 다음 단계로 전달**하는 구조가 RNN입니다.

### PyTorch로 RNN 맛보기
기본적인 입력과 출력 형태(Shape)를 알아봅시다.

In [ ]:
import torch
import torch.nn as nn

# 1. 설정값 정의
batch_size = 2      # 한 번에 처리할 문장 개수 (2개)
seq_len = 3         # 문장의 길이 (단어 3개짜리 문장)
input_size = 4      # 단어 하나를 표현하는 숫자의 개수 (Feature 차원)
hidden_size = 5     # 기억 장소(Hidden State)의 크기 (5개의 숫자로 기억)

# 2. 가짜 입력 데이터 만들기 (랜덤값)
# Shape: (배치 크기, 문장 길이, 단어 벡터 크기) -> (2, 3, 4)
x = torch.randn(batch_size, seq_len, input_size)
print("입력 데이터 형태(Shape):", x.shape)

# 3. RNN 모델 만들기
# batch_first=True: 입력의 첫 번째 차원이 Batch Size임을 명시
rnn = nn.RNN(input_size, hidden_size, batch_first=True)

# 4. 실행 (Forward)
# output: 매 시점의 출력 (기억의 기록들)
# hidden: 맨 마지막 시점의 은닉 상태 (최종 요약본)
output, hidden = rnn(x)

print("전체 출력(Output) 형태:", output.shape)  # expected: (2, 3, 5) -> (Batch, Seq, Hidden)
print("마지막 기억(Hidden) 형태:", hidden.shape) # expected: (1, 2, 5) -> (Layer, Batch, Hidden)

## 3. 실습: IMDB 영화 리뷰 감성 분석

영화 리뷰를 읽고 이 영화가 **긍정(Positive, 1)**인지 **부정(Negative, 0)**인지 맞추는 AI를 만듭니다.

- **입력**: 영화 리뷰 텍스트 (영어)
- **출력**: 0 또는 1 (이진 분류)
- **특징**: 단어의 개수를 300개(`vocab_size`)로 제한해서 실습합니다 (메모리 절약).

In [ ]:
from tensorflow.keras.datasets import imdb

vocab_size = 300  # 가장 자주 나오는 단어 300개만 쓰겠다! (나머지는 무시)

# 데이터 다운로드 및 로드
# X: 리뷰 내용(숫자 리스트), y: 정답 라벨(0/1)
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=vocab_size)

print(f"학습 데이터 개수: {len(X_train)}")
print(f"테스트 데이터 개수: {len(X_test)}")

# 데이터 조금만 쓰기 (속도를 위해)
# 학습 25000개, 테스트 10000개 그대로 사용
# 필요하다면 여기서 슬라이싱으로 줄일 수 있습니다.

### 데이터 살펴보기
컴퓨터는 글자를 모르기 때문에, 이미 **숫자(Index)**로 바뀌어 있습니다.
예를 들어 1은 '시작', 2는 'OOV' 등을 의미합니다.

In [ ]:
print("첫 번째 리뷰(숫자):", X_train[0])
print("첫 번째 리뷰 길이:", len(X_train[0]))
print("첫 번째 정답:", y_train[0]) # 1이면 긍정, 0이면 부정

### 텐서(Tensor)로 변환
딥러닝 라이브러리인 PyTorch에서 쓰려면 리스트를 `Tensor`로 바꿔야 합니다.

In [ ]:
import torch

# 리스트(List) -> 텐서(Tensor) 변환
# 하나하나 변환해서 리스트에 담습니다.
X_train = [torch.tensor(seq, dtype=torch.long) for seq in X_train]
X_test = [torch.tensor(seq, dtype=torch.long) for seq in X_test]

# 정답(Label)도 텐서로 변환 (실수형 Float)
y_train = torch.tensor(y_train, dtype=torch.float)
y_test = torch.tensor(y_test, dtype=torch.float)

print("변환 완료!")

### 패딩(Padding): 길이 맞추기
리뷰마다 길이가 제각각입니다. 어떤 건 10단어, 어떤 건 100단어...
기계학습을 위해선 길이를 똑같이 맞춰야 합니다. (직사각형 만들기)

> **비유 (테트리스)**: 빈 공간을 0으로 채워서 네모 반듯하게 만듭니다.

- 짧으면 -> 0을 채운다 (`Padding`)
- 너무 길면 -> 자른다 (`Truncating`)
- 여기선 길이를 **100**으로 통일하겠습니다.

In [ ]:
import torch.nn.functional as F

max_len = 100 # 목표 길이

def pad_sequences(sequences, max_len):
    padded_list = []
    for seq in sequences:
        if len(seq) < max_len:
            # 길이가 짧으면 뒤에 0을 채움 (pad함수: (왼쪽채움, 오른쪽채움))
            seq = F.pad(seq, (0, max_len - len(seq)), value=0)
        else:
            # 길이가 길면 앞부분부터 100개만 자름
            seq = seq[:max_len]
        padded_list.append(seq)
    # 리스트를 하나의 큰 텐서로 합침 (Stack)
    return torch.stack(padded_list)

X_train_padded = pad_sequences(X_train, max_len)
X_test_padded = pad_sequences(X_test, max_len)

print("패딩 후 형태:", X_train_padded.shape) # (25000, 100) -> 2만5천개 문장, 각 100단어

### 원-핫 인코딩 (One-Hot Encoding)
숫자 1, 2, 3은 크기 차이가 있지만, 단어 ID 1, 2, 3은 관계가 없습니다.
그래서 이를 독립적인 벡터로 바꿔줍니다.

> **주의**: 원-핫 인코딩은 0이 너무 많아져서 메모리를 많이 차지합니다. 실무에선 **임베딩(Embedding)**을 쓰지만, 여기선 원리를 배우기 위해 사용해봅니다.

In [ ]:
# (샘플수, 길이) -> (샘플수, 길이, 단어장크기)
# 300개의 칸 중 하나만 1이고 나머진 0인 벡터로 변환
X_train_onehot = F.one_hot(X_train_padded, num_classes=vocab_size).float()
X_test_onehot = F.one_hot(X_test_padded, num_classes=vocab_size).float()

print("원-핫 변환 후 형태:", X_train_onehot.shape)
# (25000, 100, 300) -> 데이터가 매우 커졌습니다!

## 4. 모델 설계 (Design)

1. **RNN 층**: 단어들을 순서대로 읽어서 문장의 의미를 압축(Hidden State)합니다.
2. **Linear 층 (FC)**: 압축된 의미를 보고 0(부정)인지 1(긍정)인지 판단 점수를 냅니다.

In [ ]:
class IMDBSentimentRNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        # 1. RNN 레이어
        self.rnn = nn.RNN(
            input_size=input_dim,   # 들어오는 벡터 크기 (300)
            hidden_size=hidden_dim, # 기억 장소 크기 (마음대로 설정, 여기선 8)
            batch_first=True        # 첫 번째 차원이 batch_size
        )
        # 2. 결론 내는 레이어 (Fully Connected)
        self.fc = nn.Linear(hidden_dim, output_dim) # 8개 특징 -> 1개 점수

    def forward(self, x):
        # x: (Batch, 100, 300)
        
        # RNN 통과
        # output: 모든 시점의 상태
        # hidden: 마지막 시점의 상태 (최종 요약본)
        output, hidden = self.rnn(x)
        
        # 우리는 '마지막 요약본'만 있으면 됩니다.
        # hidden shape: (1, Batch, Hidden) -> squeeze -> (Batch, Hidden)
        last_hidden = hidden.squeeze(0)
        
        # 점수 계산
        logit = self.fc(last_hidden)
        return logit

input_dim = vocab_size # 300
hidden_dim = 8         # 8
output_dim = 1         # 1 (긍정/부정 점수)

model = IMDBSentimentRNN(input_dim, hidden_dim, output_dim)
print(model)

## 5. 학습 준비
- **데이터 로더(DataLoader)**: 데이터를 배치(Batch) 단위로 묶어서 하나씩 꺼내줍니다.
- **손실함수(Loss)**: 틀린 정도를 계산 (BCEWithLogitsLoss: 이진 분류용)
- **최적화(Optimizer)**: 오답 노트(Loss)를 보고 모델을 수정 (Adam)

In [ ]:
from torch.utils.data import TensorDataset, DataLoader, random_split
import torch.optim as optim

# 1. 데이터셋 만들기 (입력과 정답을 묶음)
# 정답 데이터(y) 차원 맞추기: (25000) -> (25000, 1)
y_train = y_train.unsqueeze(1)
y_test = y_test.unsqueeze(1)

dataset = TensorDataset(X_train_onehot, y_train)

# 2. 학습용(Train) / 검증용(Val) 나누기 (8:2)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_ds, val_ds = random_split(dataset, [train_size, val_size])

# 3. 로더 생성 (Batch Size=64)
batch_size = 64
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False)

# 4. 장치 설정 (GPU 있으면 사용)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# 5. 도구 설정
criterion = nn.BCEWithLogitsLoss() # 이진 분류 손실함수
optimizer = optim.Adam(model.parameters(), lr=0.001) # 선생님(Adam)

print("학습 준비 완료! Device:", device)

## 6. 학습 시작! (Train Loop)
모델이 문제를 풀고, 채점 받고, 공부하는 과정을 반복합니다.

In [ ]:
from tqdm import tqdm

epochs = 5  # 교과서 5번 보기 (너무 많이 하면 시간 오래 걸림)

for epoch in range(epochs):
    model.train() # 학습 모드 전환
    train_loss = 0
    
    # tqdm으로 진행상황 바 표시
    for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
        inputs, labels = inputs.to(device), labels.to(device)
        
        # 1. 초기화 (이전 기억 지우기)
        optimizer.zero_grad()
        
        # 2. 문제 풀기 (Forward)
        outputs = model(inputs)
        
        # 3. 채점 (Loss 계산)
        loss = criterion(outputs, labels)
        
        # 4. 복습 (Backward - 기울기 계산)
        loss.backward()
        
        # 5. 수정 (Step - 파라미터 업데이트)
        optimizer.step()
        
        train_loss += loss.item()
    
    # 평균 점수 출력
    avg_loss = train_loss / len(train_loader)
    print(f"Epoch {epoch+1} 완료! 평균 손실(Loss): {avg_loss:.4f}")

## 7. 평가 (Test)
학습에 쓰지 않은(Test) 데이터로 진짜 실력을 확인합니다.

In [ ]:
model.eval() # 평가 모드 (공부 중단)
correct = 0
total = 0

test_loader = DataLoader(TensorDataset(X_test_onehot, y_test), batch_size=batch_size)

with torch.no_grad(): # 평가할 땐 수정하지 않음
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        outputs = model(inputs)
        
        # 확률로 변환 (Sigmoid)
        probs = torch.sigmoid(outputs)
        
        # 0.5보다 크면 1(긍정), 아니면 0(부정)
        predicted = (probs > 0.5).float()
        
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"테스트 정확도: {100 * correct / total:.2f}%")